In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/adware'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-02 11:55:11.199532: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 107 images belonging to 10 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(107, 10)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [5]:
X_train.shape

(74, 64, 64, 3)

In [6]:
X_test.shape

(33, 64, 64, 3)

In [7]:
y_train.shape

(74, 10)

In [8]:
y_test.shape

(33, 10)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 10

In [12]:
def malware_model():
    Malware_model = Sequential()
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    Malware_model.add(Conv2D(15, (3, 3), activation='relu'))
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    Malware_model.add(Dropout(0.25))
    Malware_model.add(Flatten())
    Malware_model.add(Dense(128, activation='relu'))
    Malware_model.add(Dropout(0.5))
    Malware_model.add(Dense(50, activation='relu'))
    Malware_model.add(Dense(num_classes, activation='softmax'))
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    #Malware_model.compile(loss='sparse_categorical_cross_entropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-02 11:55:15.060738: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-02 11:55:15.326316: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-02 11:55:15.326340: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-02 11:55:15.326769: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 11:55:15.331956: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-02 11:55:15.3322

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 15)        4065      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 15)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 15)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2940)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               3

In [16]:
y_train.shape

(74, 10)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([7, 9, 3, 3, 5, 0, 1, 9, 9, 6, 4, 3, 3, 7, 2, 4, 0, 9, 7, 3, 2, 2,
       5, 4, 5, 4, 1, 4, 5, 8, 6, 2, 8, 3, 5, 5, 6, 6, 4, 7, 3, 2, 3, 7,
       7, 3, 8, 7, 9, 6, 0, 4, 8, 7, 2, 6, 6, 1, 2, 2, 3, 5, 0, 2, 2, 3,
       4, 0, 4, 7, 4, 3, 0, 4])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,  class_weight=class_weights)
#Malware_model.add(GlobalAveragePooling2D())

2023-05-02 11:55:15.525999: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/50
3/3 [==============================] - 1s 260ms/step - loss: 0.6493 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.6082 - val_loss: 0.4834 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.4617
Epoch 2/50
3/3 [==============================] - 0s 49ms/step - loss: 0.4955 - f1_m: 0.0507 - precision_m: 0.0833 - recall_m: 0.0365 - auc: 0.6124 - val_loss: 0.3885 - val_f1_m: 0.0286 - val_precision_m: 0.1667 - val_recall_m: 0.0156 - val_auc: 0.4517
Epoch 3/50
3/3 [==============================] - 0s 50ms/step - loss: 0.4359 - f1_m: 0.1015 - precision_m: 0.2205 - recall_m: 0.0687 - auc: 0.5560 - val_loss: 0.3505 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.4650
Epoch 4/50
3/3 [==============================] - 0s 49ms/step - loss: 0.3796 - f1_m: 0.0310 - precision_m: 0.1806 - recall_m: 0.0169 - auc: 0.5855 - val_loss: 0.3412 - val_f1_m: 0.0000e+00 - val_precisio

Epoch 32/50
3/3 [==============================] - 0s 56ms/step - loss: 0.1969 - f1_m: 0.4324 - precision_m: 0.7753 - recall_m: 0.3016 - auc: 0.9308 - val_loss: 0.2630 - val_f1_m: 0.6579 - val_precision_m: 1.0000 - val_recall_m: 0.5937 - val_auc: 0.8203
Epoch 33/50
3/3 [==============================] - 0s 51ms/step - loss: 0.1685 - f1_m: 0.5616 - precision_m: 0.8839 - recall_m: 0.4138 - auc: 0.9548 - val_loss: 0.2695 - val_f1_m: 0.6579 - val_precision_m: 1.0000 - val_recall_m: 0.5937 - val_auc: 0.8156
Epoch 34/50
3/3 [==============================] - 0s 52ms/step - loss: 0.1911 - f1_m: 0.5185 - precision_m: 0.8718 - recall_m: 0.3698 - auc: 0.9550 - val_loss: 0.2551 - val_f1_m: 0.6750 - val_precision_m: 0.9375 - val_recall_m: 0.6094 - val_auc: 0.8358
Epoch 35/50
3/3 [==============================] - 0s 47ms/step - loss: 0.1710 - f1_m: 0.6751 - precision_m: 0.9232 - recall_m: 0.5323 - auc: 0.9636 - val_loss: 0.2549 - val_f1_m: 0.6628 - val_precision_m: 0.8182 - val_recall_m: 0.6094 - 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.2920 - f1_m: 0.6373 - precision_m: 0.6842 - recall_m: 0.6094 - auc: 0.8169


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 9ms/step - loss: 0.2920 - f1_m: 0.6373 - precision_m: 0.6842 - recall_m: 0.6094 - auc: 0.8169


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.6372547745704651
